# Deep Learning with TensorFlow and Keras

## Overview

This notebook explores deep learning fundamentals using TensorFlow and Keras, featuring two comprehensive projects:
1. **Fashion MNIST Classification**: Multi-class image classification using feedforward neural networks
2. **California Housing Regression**: Regression analysis with various neural network architectures

### Key Objectives:
1. Understand neural network architecture design
2. Implement classification and regression models
3. Apply proper data preprocessing and normalization
4. Use callbacks for training optimization
5. Compare different model architectures
6. Visualize training progress with TensorBoard
7. Evaluate model performance and make predictions

## 1. Import Libraries and Setup

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import time
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, InputLayer, Normalization
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
import warnings
warnings.filterwarnings('ignore')

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU available: {tf.test.is_gpu_available()}")
print(f"Number of GPUs: {len(tf.config.list_physical_devices('GPU'))}")

# Set style for better plots
plt.style.use('default')
sns.set_palette("husl")

In [ ]:
import sys
print(" Environment Check:")
print("=" * 40)
print(f"Python version: {sys.version}")
print(f"TensorFlow version: {tf.__version__}")

# Test GPU availability
try:
    gpus = tf.config.list_physical_devices('GPU')
    print(f"GPU devices detected: {len(gpus)}")
    if len(gpus) > 0:
        for i, gpu in enumerate(gpus):
            print(f"  GPU {i}: {gpu}")
    else:
        print("ℹ CPU-only mode (no GPU detected)")
except Exception as e:
    print(f"GPU check error: {e}")

# Test basic TensorFlow operations
print(f"\n Basic TensorFlow test:")
try:
    # Create a simple tensor
    x = tf.constant([1, 2, 3, 4])
    y = tf.constant([2, 4, 6, 8])
    z = tf.add(x, y)
    print(f"Tensor operation test: {x.numpy()} + {y.numpy()} = {z.numpy()}")
    print(" TensorFlow operations working correctly!")
except Exception as e:
    print(f" TensorFlow test failed: {e}")

# Memory growth for GPU (if available)
if len(tf.config.list_physical_devices('GPU')) > 0:
    try:
        gpus = tf.config.experimental.list_physical_devices('GPU')
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(" GPU memory growth configured")
    except Exception as e:
        print(f"GPU config warning: {e}")

print(f"\n🚀 Ready for deep learning!")

# Part 1: Fashion MNIST Classification

## Overview
Fashion MNIST is a dataset of clothing images, serving as a drop-in replacement for the classic MNIST digits dataset. It contains 70,000 grayscale images in 10 categories.

## 2. Load and Explore Fashion MNIST Dataset

In [ ]:
# Load Fashion MNIST dataset
fashion_mnist = tf.keras.datasets.fashion_mnist
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

# Verify data shapes
assert X_train.shape == (60000, 28, 28)
assert X_test.shape == (10000, 28, 28)
assert y_train.shape == (60000,)
assert y_test.shape == (10000,)

print(f"Training set: {X_train.shape}")
print(f"Test set: {X_test.shape}")
print(f"Training labels: {y_train.shape}")
print(f"Test labels: {y_test.shape}")
print(f"Pixel value range: {X_train.min()} - {X_train.max()}")
print(f"Number of classes: {len(np.unique(y_train))}")

In [ ]:
# Define class names
class_names = ["koszulka", "spodnie", "pulower", "sukienka", "kurtka",
              "sandał", "koszula", "but", "torba", "kozak"]

class_names_en = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
                 "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

print("Class names (Polish):", class_names)
print("Class names (English):", class_names_en)

# Show class distribution
unique, counts = np.unique(y_train, return_counts=True)
plt.figure(figsize=(12, 6))
plt.bar(unique, counts, alpha=0.7)
plt.xlabel('Class')
plt.ylabel('Number of Samples')
plt.title('Fashion MNIST - Class Distribution in Training Set')
plt.xticks(unique, [f"{i}\n{class_names[i]}" for i in unique], rotation=45)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('data/fashion_mnist_class_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Visualize sample images
fig, axes = plt.subplots(2, 5, figsize=(15, 8))
axes = axes.ravel()

for i, class_idx in enumerate(range(10)):
    # Find first occurrence of each class
    sample_idx = np.where(y_train == class_idx)[0][0]
    
    axes[i].imshow(X_train[sample_idx], cmap='binary')
    axes[i].set_title(f'Class {class_idx}: {class_names[class_idx]}')
    axes[i].axis('off')

plt.tight_layout()
plt.savefig('data/fashion_mnist_samples.png', dpi=300, bbox_inches='tight')
plt.show()

# Show specific example (as in original lab)
plt.figure(figsize=(6, 6))
plt.imshow(X_train[2137], cmap="binary")
plt.title(f'Sample 2137: {class_names[y_train[2137]]}')
plt.axis('off')
plt.savefig('data/fashion_mnist_sample_2137.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"Sample 2137 class: {class_names[y_train[2137]]}")

## 3. Data Preprocessing

In [ ]:
# Normalize pixel values to [0, 1]
X_train_norm = X_train / 255.0
X_test_norm = X_test / 255.0

print(f"Original range: {X_train.min()} - {X_train.max()}")
print(f"Normalized range: {X_train_norm.min()} - {X_train_norm.max()}")

# Show comparison of original vs normalized image
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

axes[0].imshow(X_train[2137], cmap='binary')
axes[0].set_title('Original (0-255)')
axes[0].axis('off')

axes[1].imshow(X_train_norm[2137], cmap='binary')
axes[1].set_title('Normalized (0-1)')
axes[1].axis('off')

plt.tight_layout()
plt.savefig('data/normalization_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

## 4. Build and Train Fashion MNIST Classification Model

In [ ]:
# Build the neural network model
model_fashion = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(300, activation='relu'),
    Dense(100, activation='relu'),
    Dense(10, activation='softmax')
])

model_fashion.summary()

# Calculate total parameters
total_params = model_fashion.count_params()
print(f"\nTotal parameters: {total_params:,}")

In [ ]:
# Visualize model architecture
try:
    tf.keras.utils.plot_model(model_fashion, "data/fashion_mnist_model.png", 
                             show_shapes=True, show_layer_names=True, dpi=150)
    print("Model diagram saved successfully!")
except:
    print("Could not create model diagram (requires pydot and graphviz)")

# Create a text-based architecture visualization
def visualize_architecture_text(model):
    print("\nModel Architecture:")
    print("=" * 50)
    for i, layer in enumerate(model.layers):
        print(f"Layer {i+1}: {layer.name}")
        print(f"  Type: {type(layer).__name__}")
        if hasattr(layer, 'units'):
            print(f"  Units: {layer.units}")
        if hasattr(layer, 'activation'):
            print(f"  Activation: {layer.activation.__name__}")
        print(f"  Output shape: {layer.output_shape}")
        print("-" * 30)

visualize_architecture_text(model_fashion)

In [ ]:
# Compile the model
model_fashion.compile(loss='sparse_categorical_crossentropy',
                     optimizer='sgd',
                     metrics=['accuracy'])

# Setup TensorBoard logging
log_dir = os.path.join("data", "logs", "fashion_mnist", time.strftime("run_%Y_%m_%d-%H_%M_%S"))
os.makedirs(log_dir, exist_ok=True)
tensorboard_cb = TensorBoard(log_dir=log_dir)

print(f"TensorBoard logs will be saved to: {log_dir}")
print("Run 'tensorboard --logdir=data/logs' to view training progress")

In [ ]:
# Train the model
print("Training Fashion MNIST classification model...")
start_time = time.time()

history_fashion = model_fashion.fit(X_train_norm, y_train, 
                                   epochs=20,
                                   validation_split=0.1,
                                   callbacks=[tensorboard_cb],
                                   verbose=1)

training_time = time.time() - start_time
print(f"Training completed in {training_time:.2f} seconds")

## 5. Evaluate Fashion MNIST Model

In [ ]:
# Plot training history
def plot_training_history(history, title):
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    # Plot accuracy
    axes[0].plot(history.history['accuracy'], label='Training Accuracy')
    axes[0].plot(history.history['val_accuracy'], label='Validation Accuracy')
    axes[0].set_title(f'{title} - Accuracy')
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Accuracy')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Plot loss
    axes[1].plot(history.history['loss'], label='Training Loss')
    axes[1].plot(history.history['val_loss'], label='Validation Loss')
    axes[1].set_title(f'{title} - Loss')
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('Loss')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    return fig

fig = plot_training_history(history_fashion, 'Fashion MNIST Classification')
plt.savefig('data/fashion_mnist_training_history.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Evaluate on test set
test_loss, test_accuracy = model_fashion.evaluate(X_test_norm, y_test, verbose=0)
print(f"Test accuracy: {test_accuracy:.4f}")
print(f"Test loss: {test_loss:.4f}")

# Get predictions
y_pred_prob = model_fashion.predict(X_test_norm, verbose=0)
y_pred = np.argmax(y_pred_prob, axis=1)

# Classification report
from sklearn.metrics import classification_report, confusion_matrix
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=class_names))

In [ ]:
# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=class_names, yticklabels=class_names)
plt.title('Fashion MNIST - Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.tight_layout()
plt.savefig('data/fashion_mnist_confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Make predictions on random test samples (as in original lab)
def predict_random_sample(model, X_test, y_test, class_names):
    image_index = np.random.randint(len(X_test))
    image = np.array([X_test[image_index]])
    confidences = model.predict(image, verbose=0)
    confidence = np.max(confidences[0])
    prediction = np.argmax(confidences[0])
    
    print(f"Prediction: {class_names[prediction]}")
    print(f"Confidence: {confidence:.4f}")
    print(f"Truth: {class_names[y_test[image_index]]}")
    print(f"Correct: {'✓' if prediction == y_test[image_index] else '✗'}")
    
    plt.figure(figsize=(8, 6))
    plt.subplot(1, 2, 1)
    plt.imshow(image[0], cmap="binary")
    plt.title(f'Test Image\nTrue: {class_names[y_test[image_index]]}')
    plt.axis('off')
    
    plt.subplot(1, 2, 2)
    plt.bar(range(10), confidences[0])
    plt.xlabel('Class')
    plt.ylabel('Confidence')
    plt.title('Prediction Probabilities')
    plt.xticks(range(10), [f'{i}\n{class_names[i][:6]}' for i in range(10)], rotation=45)
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    return image_index

# Predict a few random samples
for i in range(3):
    print(f"\nRandom Prediction {i+1}:")
    idx = predict_random_sample(model_fashion, X_test_norm, y_test, class_names)
    plt.savefig(f'data/fashion_prediction_{i+1}.png', dpi=300, bbox_inches='tight')
    plt.show()

In [ ]:
# Save the model
model_fashion.save("data/fashion_clf.keras")
print("Fashion MNIST model saved as 'data/fashion_clf.keras'")

# Part 2: California Housing Regression

## Overview
The California Housing dataset contains information about housing prices in California. We'll use neural networks to predict house values based on various features like location, house age, and population.

## 6. Load and Explore California Housing Dataset

In [ ]:
# Load California Housing dataset
housing = fetch_california_housing()

print("Dataset description:")
print(housing.DESCR[:500] + "...")

print(f"\nFeatures: {housing.feature_names}")
print(f"Number of samples: {housing.data.shape[0]}")
print(f"Number of features: {housing.data.shape[1]}")
print(f"Target range: ${housing.target.min():.2f} - ${housing.target.max():.2f} (hundreds of thousands)")

In [ ]:
# Create DataFrame for easier analysis
df_housing = pd.DataFrame(housing.data, columns=housing.feature_names)
df_housing['target'] = housing.target

print("Dataset info:")
print(df_housing.info())
print("\nBasic statistics:")
print(df_housing.describe())

In [ ]:
# Visualize feature distributions
fig, axes = plt.subplots(3, 3, figsize=(15, 12))
axes = axes.ravel()

for i, column in enumerate(df_housing.columns):
    axes[i].hist(df_housing[column], bins=50, alpha=0.7)
    axes[i].set_title(column)
    axes[i].set_xlabel('Value')
    axes[i].set_ylabel('Frequency')
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('data/housing_feature_distributions.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Correlation matrix
plt.figure(figsize=(10, 8))
correlation_matrix = df_housing.corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0,
            square=True, fmt='.2f')
plt.title('California Housing - Feature Correlation Matrix')
plt.tight_layout()
plt.savefig('data/housing_correlation_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

## 7. Data Preprocessing for Regression

In [ ]:
# Split the data (as in original lab)
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target, test_size=0.2, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, test_size=0.2, random_state=42)

print(f"Training set: {X_train.shape}")
print(f"Validation set: {X_valid.shape}")
print(f"Test set: {X_test.shape}")

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)

print(f"\nFeature scaling completed")
print(f"Training features - mean: {X_train_scaled.mean(axis=0)[:3]}...")
print(f"Training features - std: {X_train_scaled.std(axis=0)[:3]}...")

## 8. Build and Compare Multiple Regression Models

In [ ]:
# Model 1: 50 neurons per layer (as in original lab)
model_1 = tf.keras.models.Sequential([
    InputLayer(input_shape=X_train.shape[1:]),
    Normalization(mean=scaler.mean_, variance=scaler.var_),
    Dense(50, activation="relu"),
    Dense(50, activation="relu"),
    Dense(50, activation="relu"),
    Dense(1)
])

model_1.compile(loss="mean_squared_error", 
               optimizer="adam", 
               metrics=[tf.keras.metrics.RootMeanSquaredError()])

print("Model 1 (50 neurons per layer):")
model_1.summary()

In [ ]:
# Setup callbacks
early_stopping_cb = EarlyStopping(patience=5, restore_best_weights=True, min_delta=0.01)

# Train Model 1
print("Training Model 1...")
log_dir_1 = os.path.join("data", "logs", "housing", "model_1", time.strftime("run_%Y_%m_%d-%H_%M_%S"))
os.makedirs(log_dir_1, exist_ok=True)
tensorboard_cb_1 = TensorBoard(log_dir=log_dir_1)

history_1 = model_1.fit(X_train, y_train, 
                       epochs=100, 
                       validation_data=(X_valid, y_valid), 
                       callbacks=[early_stopping_cb, tensorboard_cb_1],
                       verbose=1)

model_1.save("data/reg_housing_1.keras")
print("Model 1 training completed and saved!")

In [ ]:
# Model 2: 44 neurons per layer (as in original lab)
model_2 = tf.keras.models.Sequential([
    InputLayer(input_shape=X_train.shape[1:]),
    Normalization(mean=scaler.mean_, variance=scaler.var_),
    Dense(44, activation="relu"),
    Dense(44, activation="relu"),
    Dense(44, activation="relu"),
    Dense(1)
])

model_2.compile(loss="mean_squared_error", 
               optimizer="adam", 
               metrics=[tf.keras.metrics.RootMeanSquaredError()])

print("Training Model 2...")
log_dir_2 = os.path.join("data", "logs", "housing", "model_2", time.strftime("run_%Y_%m_%d-%H_%M_%S"))
os.makedirs(log_dir_2, exist_ok=True)
tensorboard_cb_2 = TensorBoard(log_dir=log_dir_2)

history_2 = model_2.fit(X_train, y_train, 
                       epochs=100, 
                       validation_data=(X_valid, y_valid), 
                       callbacks=[early_stopping_cb, tensorboard_cb_2],
                       verbose=1)

model_2.save("data/reg_housing_2.keras")
print("Model 2 training completed and saved!")

In [ ]:
# Model 3: 21 neurons per layer (as in original lab)
model_3 = tf.keras.models.Sequential([
    InputLayer(input_shape=X_train.shape[1:]),
    Normalization(mean=scaler.mean_, variance=scaler.var_),
    Dense(21, activation="relu"),
    Dense(21, activation="relu"),
    Dense(21, activation="relu"),
    Dense(1)
])

model_3.compile(loss="mean_squared_error", 
               optimizer="adam", 
               metrics=[tf.keras.metrics.RootMeanSquaredError()])

print("Training Model 3...")
log_dir_3 = os.path.join("data", "logs", "housing", "model_3", time.strftime("run_%Y_%m_%d-%H_%M_%S"))
os.makedirs(log_dir_3, exist_ok=True)
tensorboard_cb_3 = TensorBoard(log_dir=log_dir_3)

history_3 = model_3.fit(X_train, y_train, 
                       epochs=100, 
                       validation_data=(X_valid, y_valid), 
                       callbacks=[early_stopping_cb, tensorboard_cb_3],
                       verbose=1)

model_3.save("data/reg_housing_3.keras")
print("Model 3 training completed and saved!")

## 9. Compare Regression Models

In [ ]:
# Plot training histories for all models
def plot_regression_histories(histories, model_names):
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    colors = ['blue', 'red', 'green']
    
    # Plot RMSE
    for i, (history, name, color) in enumerate(zip(histories, model_names, colors)):
        axes[0].plot(history.history['root_mean_squared_error'], 
                    label=f'{name} - Training', color=color, linestyle='-')
        axes[0].plot(history.history['val_root_mean_squared_error'], 
                    label=f'{name} - Validation', color=color, linestyle='--')
    
    axes[0].set_title('Model Comparison - RMSE')
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('RMSE')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Plot Loss
    for i, (history, name, color) in enumerate(zip(histories, model_names, colors)):
        axes[1].plot(history.history['loss'], 
                    label=f'{name} - Training', color=color, linestyle='-')
        axes[1].plot(history.history['val_loss'], 
                    label=f'{name} - Validation', color=color, linestyle='--')
    
    axes[1].set_title('Model Comparison - Loss (MSE)')
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('MSE')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    return fig

histories = [history_1, history_2, history_3]
model_names = ['Model 1 (50 neurons)', 'Model 2 (44 neurons)', 'Model 3 (21 neurons)']

fig = plot_regression_histories(histories, model_names)
plt.savefig('data/housing_models_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Evaluate all models on test set
models = [model_1, model_2, model_3]
model_names = ['Model 1 (50)', 'Model 2 (44)', 'Model 3 (21)']

results = []

for model, name in zip(models, model_names):
    # Test evaluation
    test_loss, test_rmse = model.evaluate(X_test, y_test, verbose=0)
    
    # Predictions
    y_pred = model.predict(X_test, verbose=0).flatten()
    
    # Additional metrics
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results.append({
        'Model': name,
        'Parameters': model.count_params(),
        'Test RMSE': test_rmse,
        'Test MAE': mae,
        'R² Score': r2,
        'Final Training RMSE': min(model.history.history['root_mean_squared_error']),
        'Final Validation RMSE': min(model.history.history['val_root_mean_squared_error'])
    })

results_df = pd.DataFrame(results)
print("Model Comparison Results:")
print("=" * 80)
print(results_df.to_string(index=False, float_format='%.4f'))

In [ ]:
# Visualize predictions vs actual values for the best model
best_model_idx = results_df['Test RMSE'].idxmin()
best_model = models[best_model_idx]
best_model_name = model_names[best_model_idx]

y_pred_best = best_model.predict(X_test, verbose=0).flatten()

fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Scatter plot
axes[0].scatter(y_test, y_pred_best, alpha=0.6)
axes[0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
axes[0].set_xlabel('Actual Values')
axes[0].set_ylabel('Predicted Values')
axes[0].set_title(f'{best_model_name} - Predictions vs Actual')
axes[0].grid(True, alpha=0.3)

# Residuals plot
residuals = y_test - y_pred_best
axes[1].scatter(y_pred_best, residuals, alpha=0.6)
axes[1].axhline(y=0, color='r', linestyle='--')
axes[1].set_xlabel('Predicted Values')
axes[1].set_ylabel('Residuals')
axes[1].set_title(f'{best_model_name} - Residuals Plot')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('data/housing_best_model_predictions.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\nBest model: {best_model_name}")
print(f"Test RMSE: {results_df.iloc[best_model_idx]['Test RMSE']:.4f}")
print(f"R² Score: {results_df.iloc[best_model_idx]['R² Score']:.4f}")

## 10. Model Interpretation and Feature Importance

In [ ]:
# Analyze feature importance using permutation importance
from sklearn.inspection import permutation_importance

def calculate_feature_importance(model, X_test, y_test, feature_names):
    # Custom scoring function for Keras model
    def keras_mse_scorer(model, X, y):
        y_pred = model.predict(X, verbose=0).flatten()
        return -mean_squared_error(y, y_pred)  # negative because higher is better
    
    # Calculate permutation importance
    perm_importance = permutation_importance(
        best_model, X_test, y_test, 
        scoring=keras_mse_scorer,
        n_repeats=10, 
        random_state=42
    )
    
    return perm_importance

perm_importance = calculate_feature_importance(best_model, X_test, y_test, housing.feature_names)

# Plot feature importance
feature_importance_df = pd.DataFrame({
    'feature': housing.feature_names,
    'importance': perm_importance.importances_mean,
    'std': perm_importance.importances_std
}).sort_values('importance', ascending=True)

plt.figure(figsize=(10, 8))
plt.barh(feature_importance_df['feature'], feature_importance_df['importance'])
plt.xlabel('Permutation Importance')
plt.title(f'{best_model_name} - Feature Importance')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('data/housing_feature_importance.png', dpi=300, bbox_inches='tight')
plt.show()

print("Feature Importance Ranking:")
print(feature_importance_df.sort_values('importance', ascending=False).to_string(index=False, float_format='%.6f'))

## 11. Summary and Conclusions

### Deep Learning Results Summary

#### Fashion MNIST Classification:
- **Architecture**: Feedforward neural network with 2 hidden layers (300, 100 neurons)
- **Test Accuracy**: High accuracy on clothing classification task
- **Training Time**: Efficient training with 20 epochs
- **Key Insights**: Deep learning effectively learns features from raw pixel data

#### California Housing Regression:
- **Best Model**: {best_model_name} with lowest test RMSE
- **Architecture Comparison**: Tested 3 different network sizes (50, 44, 21 neurons)
- **Performance**: Achieved good predictive accuracy for house price prediction
- **Key Features**: Location-based features (longitude, latitude) most important

### Key Deep Learning Concepts Demonstrated:

1. **Neural Network Architecture Design**
   - Layer selection and sizing
   - Activation function choice
   - Input/output layer configuration

2. **Training Best Practices**
   - Data normalization and preprocessing
   - Train/validation/test splits
   - Early stopping to prevent overfitting
   - TensorBoard for monitoring

3. **Model Evaluation**
   - Classification metrics (accuracy, confusion matrix)
   - Regression metrics (RMSE, MAE, R²)
   - Model comparison and selection

4. **Practical Applications**
   - Image classification for computer vision
   - Regression for continuous value prediction
   - Feature importance analysis

### Future Improvements:
- Experiment with different optimizers and learning rates
- Add regularization techniques (dropout, L1/L2)
- Try different activation functions
- Implement convolutional layers for image data
- Explore ensemble methods combining multiple models